#### Qlunc tutorial by Francisco Costa
# **Tutorial 2**
## Calculate line of sight wind velocity ($V_{LOS}$) and horizontal wind velocity ($V_{h}$) uncertainties

This tutorial aims to facilitate the use to Qlunc. 
In this tutorial will deal with the new capabilities of Qlunc, namely $V_{LOS}$ (one single lidar) and $V_{h}$ (two lidars) estimations. To quantitatively verify the results a comparison between Montecarlo simulations and the analytical model (GUM approach) is performed.

* **Some steps are not deeply explained since it is not necessary for the sake of this tutorial. The main goal of the tutorial is to show how the lidar digital twin is built and the interaction between different modules.**

In [1]:
import os
os.chdir('../')
# importing  uncertainty functions
import UQ_Functions.UQ_Vhorizontal_Classes as uVhc, UQ_Functions.UQ_Photonics_Classes as uphc,UQ_Functions.UQ_Optics_Classes as uopc, UQ_Functions.UQ_Power_Classes as upwc,UQ_Functions.UQ_Lidar_Classes as ulc, UQ_Functions.UQ_ProbeVolume_Classes as upbc,UQ_Functions.UQ_Data_processing_Classes as uprm, UQ_Functions.UQ_SignalProcessor_Classes as uspc
from Utils.Qlunc_ImportModules import *

A [.yml template](https://github.com/PacoCosta/Qlunc/blob/Qlunc-V0.9/Main/Template_yaml_inputs_file.yml) is included in the repository. This tutorial uses two different yaml files to create two lidars 'Caixa1' and 'Caixa2'. 

The code below imports the `yaml` file.

In [2]:
# Read yaml files 1 and 2
# Obtain data from .yaml file:
with open (r'./Tutorials/yaml_inputs_file_2_1.yml','r') as file:
    Qlunc_yaml_inputs1={}
    docs = yaml.load_all(file, Loader=yaml.FullLoader)
    for doc in docs:      
        for k, v in doc.items():     
            Qlunc_yaml_inputs1.setdefault(k,v)
# Obtain data from .yaml file:
with open (r'./Tutorials/yaml_inputs_file_2_2.yml','r') as file:
    Qlunc_yaml_inputs2={}
    docs = yaml.load_all(file, Loader=yaml.FullLoader)
    for doc in docs:      
        for k, v in doc.items():     
            Qlunc_yaml_inputs2.setdefault(k,v)

In [3]:
# Execute Qlunc_Classes.py (creating classes for lidar 'objects'):
#exec(open(Qlunc_yaml_inputs1['Main directory']+'/Tutorials/WorkingExample_Tutorial.py').read())

In [4]:
!dir #print(Qlunc_yaml_inputs1)

 Volume in drive C has no label.
 Volume Serial Number is 0C54-9785

 Directory of C:\SWE_LOCAL\GIT_Qlunc



File Not Found


Bla bla bla ...

In [5]:
%run -i ./Tutorials/Qlunc_Instantiate_Tutorial2_file_1.py
%run -i ./Tutorials/Qlunc_Instantiate_Tutorial2_file_2.py

C:\SWE_LOCAL\GIT_Qlunc
C:\SWE_LOCAL\GIT_Qlunc
Created new scanner: Scanner_Yaml
Created new optical circulator: OC_Yaml
Created new telescope: Telescope_YAML_zr
Class "Probe volume" created
Created new optic module: Optics_module_Yaml
Created new AOM: AOM_YAML
Created new optical amplifier: OA_Yaml
Created new photodetector: Photodetector_YAML
Created new laser: Laser_YAML
Created new photonic module: Photonics_module_Yaml
Created new ADC: ADC_Yaml
Created new signal processor module: Signal_processor_module_Yaml
Created new lidar general inputs: Lidar_gral_inputs_Yaml
Selected wfr model: None terrain
Selected filtering model: CNR 
Created new lidar device: Caixa4
Created new atmosphere: Atmosphere_Yaml
